In [26]:
#Dependencies
import os
import csv
import pandas as pd

In [37]:
# Store the file path associated with the file
csvpath = os.path.join('.', 'breweries.csv')
data_df = pd.read_csv(csvpath)

#Create empty lists to store data we want
brewery_name = []
brewery_type = []
brewery_city = []
brewery_state = []
brewery_county = []
brewery_country = []

brew_df = data_df[["name", "brewery_type", 'city', "state", 'country']]
brew_df.describe()

renamed_df = brew_df.rename(columns={"name":"Brewery Name", "brewery_type":"Type of Brewery",
                                    "city": "City", "state": "State", "country": "Country"})
renamed_df.head()

,Brewery Name,Type of Brewery,City,State,Country
0,10-56 Brewing Company,micro,Knox,Indiana,United States
1,10 Barrel Brewing Co,large,Bend,Oregon,United States
2,10 Barrel Brewing Co,large,Bend,Oregon,United States
3,10 Barrel Brewing Co - Bend Pub,large,Bend,Oregon,United States
4,10 Barrel Brewing Co - Boise,large,Boise,Idaho,United States


In [28]:
#Read csv file
with open(csvpath) as csvfile:

    # CSV reader specifies delimiter and variable that holds contents
    csvreader = csv.reader(csvfile, delimiter=',')

    print(csvreader)
    print('--------')

    # Read the header row first (skip this step if there is now header)
    csv_header = next(csvreader)
    print(f"CSV Header: {csv_header}")
    
    for row in csvreader:
        brewery_name.append(row[1])
        brewery_type.append(row[2])
        brewery_city.append(row[6])
        brewery_state.append(row[7])
        brewery_county.append(row[8])
        brewery_country.append(row[14])


--------
CSV Header: ['id', 'name', 'brewery_type', 'street', 'address_2', 'address_3', 'city', 'state', 'county_province', 'postal_code', 'website_url', 'phone', 'created_at', 'updated_at', 'country', 'longitude', 'latitude', 'tags']


In [29]:
#Store data as Data Frame
brewery_df = pd.DataFrame("Name": brewery_name, 
                          "Type": brewery_type, 
                          "City": brewery_city,
                          "State": brewery_state,
                          "County": brewery_county,
                          "Country": brewery_country)

SyntaxError: invalid syntax (<ipython-input-29-892587d70dd0>, line 2)